# Vehicle Detection and Tracking
In this project, I will have to write a software pipeline to identify vehicles in a video from a front-facing camera on a car.


## Python Imports 
We will start by importing the various libs we need.
I like to denormalize so 2 seperate files were created `detector.py` with a class called Detector with some HOG features and configuration, and another file named `util.py` with some useful helper functions taken from the classroom at Udacity.

In [2]:
import cv2
import numpy as np
import os
import argparse
import pickle
from util import extract_features, rgb, slide_window, draw_boxes, make_heatmap, get_hog_features, color_hist, bin_spatial
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.measurements import label as label_image
from skimage.filters.rank import windowed_histogram
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

# import the detector class in the detector class
from detector import Detector

import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

# Get some helper functions from the util file containing the functions taken in the classroom
from util import read_image, draw_boxes, rgb, write_image, convert_video_frame, make_heatmap
import time

# Train Classifier
For this project, I chose to use a linear Support Vector Machine (SVM). Various background research has shown that SVM is a good compliment to HOG features.

In [ ]:
try:
    from sklearn.model_selection import train_test_split
except:
    from sklearn.cross_validation import train_test_split
from util import extract_features_from_images
import yaml

non_vehicle_directory = './non-vehicles'
vehicle_directory = './vehicles'

def find_images(directory):
    for root, dirs, files in os.walk(directory):
        for f in files:
            if f.split('.')[-1].lower() in ['jpg', 'png']:
                yield os.path.join(root, f)


feature_parameters = {
    'cspace': 'YCrCb',
    'spatial_size': (32, 32),
    'hist_bins': 32,
    'hog_orient': 9,
    'hog_pix_per_cell': 8,
    'hog_cell_per_block': 2,
    'hog_channel': 'ALL'
}

print("Loading vehicle images and extracting features...")
vehicle_features = extract_features_from_images(find_images(vehicle_directory),
                                                **feature_parameters)
print("Extracted features from {} vehicle images".format(len(vehicle_features)))

print("Loading non-vehicle images and extracting features...")
non_vehicle_features = extract_features_from_images(find_images(non_vehicle_directory),
                                                    **feature_parameters)
print("Extracted features from {} non-vehicle images".format(len(non_vehicle_features)))

X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Training the data using SVM
print("Training...")
svc = LinearSVC()
svc.fit(X_train, y_train)

positives = y_test.nonzero()
negatives = np.logical_not(y_test).nonzero()
tp = svc.score(X_test[positives], y_test[positives])
tn = svc.score(X_test[negatives], y_test[negatives])

# Get the positive and negative rates
print('True Positive Rate:  {:.2f}%'.format(100*tp))
print('True Negative Rate:  {:.2f}%'.format(100*tn))
print('False Positive Rate: {:.2f}%'.format(100*(1-tn)))
print('False Negative Rate: {:.2f}%'.format(100*(1-tp)))

# Save the classifier trained into the pickle file "classifier.p"
print('Pickling classifier to classifier.p')
with open('classifier.p', 'wb') as f:
    data = {
        'feature_parameters': feature_parameters,
        'classifier': svc,
        'shape': (64, 64),
        'scaler': scaler
    }
    pickle.dump(data, f)

Loading vehicle images and extracting features...
Extracted features from 8792 vehicle images
Loading non-vehicle images and extracting features...
Extracted features from 8968 non-vehicle images
Training...
True Positive Rate:  99.16%
True Negative Rate:  99.15%
False Positive Rate: 0.85%
False Negative Rate: 0.84%
Pickling classifier to classifier.p


# Detecting Vehicles and Saving Results
The vehicles on the lane are detected using the detected class functions of the `detector.py` file.
Once these cars are detected, the output is done on the fly to either the video or image fil

In [18]:
# Useful variables
## Image or video file to process
input_file = './test_images/test4.jpg'

## Output file with boxes drawn
output_file = './output_images/test4.jpg'

## Tweaking parameters 
### (Heatmap value to activate & alpha value for heatmap smoothing filter)
smoothing = 0.125
heat_threshold = 2.25

## Beginning and end times of video
subclip = []

# Get the previously classifier pickle file
print('Loading classifier from pickle classifier.p')
with open('classifier.p', 'rb') as f:
    data = pickle.load(f)
    classifier = data['classifier']
    feature_parameters = data['feature_parameters']
    window_shape = data['shape']
    scaler = data['scaler']

print('Feature parameters:')
print(feature_parameters)

file_extension = input_file.split('.')[-1].lower()

if file_extension in ['jpg', 'png']:
    detector = Detector(classifier, feature_parameters, window_shape, scaler, heat_threshold)

    print('Loading ' + input_file + ' as a ' + feature_parameters['cspace'] + ' image')
    img = read_image(input_file, feature_parameters['cspace'])
    output_to_file = output_file and len(output_file)

    print('Detecting vehicles')
    boxes = detector(img, show_plots=(not output_to_file))
    print(boxes)
    output = draw_boxes(rgb(img, feature_parameters['cspace']), boxes)

    if output_to_file:
        print('Writing output to ' + output_file)
        write_image(output_file, output, 'RGB')
    else:
        plt.figure()
        plt.title(input_file)
        plt.imshow(output)
        plt.show()

elif file_extension in ['mp4']:
    detector = Detector(classifier, feature_parameters, window_shape, scaler, heat_threshold, alpha=smoothing)

    def frame_handler(frame):
        boxes = detector(convert_video_frame(frame, feature_parameters['cspace']))
        output = draw_boxes(frame, boxes)
        return output

    clip = VideoFileClip(input_file)

    if (len(subclip) > 0):
        clip = clip.subclip(subclip[0], subclip[1])
    clip = clip.fl_image(frame_handler)

    print("Writing video file to {}".format(output_file))
    clip.write_videofile(output_file, audio=False)
    print("Done")
else:
    raise Exception('Unidentified file extension' + file_extension)

Loading classifier from pickle classifier.p
Feature parameters:
{'spatial_size': (32, 32), 'hog_channel': 'ALL', 'hog_cell_per_block': 2, 'hog_pix_per_cell': 8, 'cspace': 'YCrCb', 'hist_bins': 32, 'hog_orient': 9}
Loading ./test_images/test4.jpg as a YCrCb image
Detecting vehicles
[((812, 376), (951, 513)), ((1072, 388), (1247, 527))]
Writing output to ./output_images/test4.jpg
